Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [26]:
import numpy as np

In [42]:
NUM_KNAPSACKS = 2
NUM_ITEMS = 10
NUM_DIMENSIONS = 2

In [43]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS)
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = np.random.randint(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)

In [44]:
CONSTRAINTS

array([476, 303])

In [45]:
# A random solution
solution = np.array(
    [np.random.random(NUM_ITEMS) < 0.5 for _ in range(NUM_KNAPSACKS)], dtype=bool
)

In [46]:
solution

array([[ True, False,  True,  True,  True, False,  True, False,  True,
         True],
       [ True, False, False, False, False, False, False, False,  True,
        False]])

In [47]:
# Check that the same object does not appear in multiple knapsacks
np.all(solution.sum(axis=0) <= 1)

False

In [48]:
# Check if the solution is valid
all_knapsacks = np.any(solution, axis=0)
np.all(WEIGHTS[all_knapsacks].sum(axis=0) < CONSTRAINTS)

False

## HELPER FUNCTIONS

In [49]:
def total_value(solution):
    picked_items = np.any(solution, axis=0)
    return VALUES[picked_items].sum()

In [50]:
total_value(solution)

422

In [51]:
def is_feasible_per_knapsack(solution, weights, constraints):
    """
    Returns True if every knapsack k satisfies that: sum of the weights in k <= constraints.
    """
    K, N = solution.shape # number of knapsacks, number of items
    for k in range(K):
        items_k = solution[k]
        if items_k.any():
            load_k = weights[items_k].sum(axis=0)          # total load on the knapsack, shape (D,)
            if np.any(load_k > constraints):
                return False
    # also enforce at most one knapsack per item: 
    if np.any(solution.sum(axis=0) > 1):
        return False
    return True

In [52]:
# test random solution
is_feasible_per_knapsack(solution, WEIGHTS, CONSTRAINTS)

False

In [55]:
NUM_KNAPSACKS = 2
NUM_ITEMS = 4
NUM_DIMENSIONS = 2

test_weights = np.array([
    [3, 5],
    [7, 1],   
    
    [4, 2],  
    [2, 6]   
])
test_constraints = np.array([10, 10])

# Suppose each knapsack has 2 items
test_solution = np.array([
    [1, 1, 0, 0],  
    [0, 0, 1, 1]   
], dtype=bool)


In [56]:
# test valid solution
is_feasible_per_knapsack(test_solution, test_weights, test_constraints)

True

## TEST PROBLEMS

In [23]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)

In [68]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)

In [69]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)